In [36]:
from pathlib import Path

import yfinance as yf
import pandas as pd
import plotly.io as pio
import plotly.graph_objects as go

In [29]:
import plotly.express as px

pio.renderers.default = "browser"

In [30]:
# Load S&P 500 data
csv_file = Path("sp500.csv")
if not csv_file.exists():
    df = yf.download("^GSPC")

    df = df.reset_index()  # Use an integer index
    df.columns = df.columns.droplevel(1)  # Remove the second level (ticker)
    df.to_csv(csv_file)
else:
    df = pd.read_csv(csv_file, parse_dates=True)

In [47]:
# Calculate the running maximum price
df["Running Max"] = df["Close"].cummax()
df["Drawdown"] = (df["Close"] - df["Running Max"]) / df["Running Max"]

# Create a column for plotting downturns: Copy 'Close' value if drawdown <= -0.10, else NaN
df["Downturn Close"] = df.loc[df["Drawdown"] <= -0.10, "Close"]
df["Normal Close"] = df.loc[df["Drawdown"] > -0.10, "Close"]

downturn_days = df[df["Drawdown"] <= -0.10]
downturn_days = downturn_days[["Date", "Close", "Running Max", "Drawdown"]]

downturn_days

,Date,Close,Running Max,Drawdown
112,1928-06-12,18.340000,20.440001,-0.102740
117,1928-06-19,18.340000,20.440001,-0.102740
442,1929-10-04,28.650000,31.860001,-0.100753
453,1929-10-21,27.730000,31.860001,-0.129630
454,1929-10-22,28.270000,31.860001,-0.112680
...,...,...,...,...
24441,2025-04-22,5287.759766,6144.149902,-0.139383
24442,2025-04-23,5375.859863,6144.149902,-0.125044
24443,2025-04-24,5484.770020,6144.149902,-0.107318
24444,2025-04-25,5525.209961,6144.149902,-0.100736


In [67]:
fig = px.line(
    df,
    x="Date",
    y="Normal Close",
    title="S&P 500 Closing Price (Log Scale) with >10% Downturns",
    log_y=True,
)
fig.add_trace(
    go.Scatter(
        x=df["Date"],
        y=df["Downturn Close"], 
        mode="lines",
        line=dict(
            color="red",
            width=1,
        ), 
        name=">10% Downturn",  
    )
)
fig.update_layout(legend=dict(yanchor="top", y=0.99, xanchor="left", x=0.01))


fig.show()